#### Package 설치

In [1]:
%pip install -q langchain-text-splitters docx2txt langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

#### 1. Knowledge Base 구성을 위한 데이터 생성

- [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)를 활용한 데이터 chunking
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
    - LangChain에서 다양한 [TextSplitter](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)들을 제공
- `chunk_size` 는 split 된 chunk의 최대 크기
- `chunk_overlap`은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('data/tax_with_table.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

print(len(document_list))
print(type(document_list[0]))
print(document_list[:2])

225
<class 'langchain_core.documents.base.Document'>
[Document(metadata={'source': 'data/tax_with_table.docx'}, page_content='소득세법\n\n소득세법\n\n[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]\n\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n\n기획재정부(소득세제과(근로소득)) 044-215-4216\n\n기획재정부(재산세제과(양도소득세)) 044-215-4314\n\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236\n\n\n\n\t제1장 총칙 <개정 2009. 12. 31.>\t\n\n\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n\n[본조신설 2009. 12. 31.]\n\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n\n[본조신설 2009. 12. 31.]\n\n\n\n제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진

In [4]:
from langchain_openai import OpenAIEmbeddings

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
print(embedding)

client=<openai.resources.embeddings.Embeddings object at 0x000002106E2116D0> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000002106CF4B290> model='text-embedding-3-large' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True


In [ ]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./db/chroma")

# 이미 저장된 데이터를 사용할 때 
#database = Chroma(collection_name='chroma-tax', persist_directory="./db/chroma", embedding_function=embedding)
print(database)

#### 2. 답변 생성을 위한 Retrieval

- `Chroma`에 저장한 데이터를 유사도 검색(`similarity_search()`)를 활용해서 가져옴

In [6]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

print(len(retrieved_docs))
print(type(retrieved_docs[0]))
print(retrieved_docs[0].metadata)

3
<class 'langchain_core.documents.base.Document'>
{'source': 'data/tax_with_table.docx'}


In [7]:

print(retrieved_docs[0].page_content[:100])

나. 그 밖의 배당소득에 대해서는 100분의 14

3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 


#### 3. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [9]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


#### 4. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [13]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
ai_message = qa_chain.invoke({"query": query})
print(ai_message)

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?', 'result': '5천만원인 직장인의 연봉 소득세율은 100분의 3이며, 근로소득에 대해서는 기본세율이 적용됩니다. 그 외의 기타소득 및 소득구분에 따라 세율이 다르게 적용될 수 있습니다.'}


In [14]:
query = '비과세소득에 어떤 것들이 있나요?'

ai_message = qa_chain.invoke({"query": query})
print(ai_message)

{'query': '비과세소득에 어떤 것들이 있나요?', 'result': '비과세소득은 국군포로가 받는 위로지원금과 그 밖의 금품, 국가지정문화재로 지정된 서화ㆍ골동품의 양도로 발생하는 소득, 서화ㆍ골동품을 박물관 또는 미술관에 양도함으로 발생하는 소득 등이 있습니다.'}


* LangChain 기반의 RAG(Retrieval-Augmented Generation) 파이프라인을 구현하여 DOCX 문서를 로드, 임베딩, 검색, 그리고 LLM을 통한 답변 생성

In [17]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import hub
from langchain.document_loaders import Docx2txtLoader
import warnings

warnings.filterwarnings("ignore", category=UserWarning)  # 특정 경고 유형만 무시

#  1. 환경 변수 로드
load_dotenv()

#  2. OpenAI API 키 확인
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError(" OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")

#  3. DOCX 파일 로드 및 텍스트 추출 (Docx2txtLoader 활용)
def load_docx(file_path):
    """DOCX 파일에서 텍스트를 추출하는 함수."""
    try:
        loader = Docx2txtLoader(file_path)
        documents = loader.load()
        text = "\n".join([doc.page_content for doc in documents])
        if not text.strip():
            raise ValueError(" 문서에서 텍스트를 추출할 수 없습니다.")
        return text
    except Exception as e:
        raise RuntimeError(f" 문서 로딩 실패: {str(e)}")

#  4. 문서 분할 함수
def split_text(text, chunk_size=500, chunk_overlap=50):
    """텍스트를 지정된 크기의 청크로 분할하는 함수."""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    return splitter.split_text(text)

#  5. 벡터 데이터베이스(Chroma) 생성 함수
def create_vector_store(text_chunks, embedding_model):
    """텍스트 청크를 임베딩하고 Chroma 벡터 저장소에 저장."""
    try:
        documents = [Document(page_content=chunk) for chunk in text_chunks]
        vector_store = Chroma.from_documents(documents, embedding_model)
        return vector_store
    except Exception as e:
        raise RuntimeError(f"❌ 벡터 저장소 생성 실패: {str(e)}")

#  6. LLM을 활용한 질문 응답 함수
def query_with_llm(query, vector_store):
    """LLM을 사용하여 검색된 문서 기반으로 답변 생성."""
    try:
        # LLM 모델 설정
        llm = ChatOpenAI(model_name="gpt-3.5-turbo")
        
        # 프롬프트 로드 (RAG 최적화된 LangChain Hub 프롬프트 사용)
        prompt = hub.pull("rlm/rag-prompt")

        # RetrievalQA 체인 생성
        qa_chain = RetrievalQA.from_chain_type(
            llm, 
            retriever=vector_store.as_retriever(),
            chain_type_kwargs={"prompt": prompt}
        )

        # LLM 응답 생성
        ai_message = qa_chain.invoke({"query": query})
        print(ai_message)
        return ai_message["result"]

    except Exception as e:
        raise RuntimeError(f" LLM 응답 생성 실패: {str(e)}")

#  실행 예제
if __name__ == "__main__":
    # DOCX 파일 경로
    docx_path = "data/tax_with_table.docx"
    
    # 1. 문서 로드
    text = load_docx(docx_path)
    print(" 문서 로드 완료")
    
    # 2. 문서 분할
    text_chunks = split_text(text)
    print(f" 문서 분할 완료: {len(text_chunks)}개 청크 생성")
    
    # 3. 임베딩 모델 초기화
    embedding_model = OpenAIEmbeddings()
    
    # 4. 벡터 저장소 생성
    vector_store = create_vector_store(text_chunks, embedding_model)
    print(" 벡터 저장소 생성 완료")
    
    # 5. 질의 실행 (검색만 수행)
    #query = "소득세법이란?"
    query = "총수입금액 불산입에 대하여 설명해 주세요."
    results = query_with_llm(query, vector_store)
    
    # 6. AI 응답 출력
    print("\n🔹 AI의 답변:")
    print(results)


 문서 로드 완료
 문서 분할 완료: 720개 청크 생성
 벡터 저장소 생성 완료
{'query': '총수입금액 불산입에 대하여 설명해 주세요.', 'result': '총수입금액 불산입이란, 개별소비세, 주세, 부가가치세 및 특례제한법에 따른 세액이 해당 과세기간의 소득금액 계산 시 총수입금액에 반영되지 않는 것을 말합니다. 이는 국세환급가산금, 지방세환급가산금, 이자, 석유판매업자의 세금환급과 같은 경우도 해당됩니다. 이는 세금 환급, 무상으로 받은 자산의 가액 또는 이월된 소득금액을 고려할 때 중요한 사항입니다.'}

🔹 AI의 답변:
총수입금액 불산입이란, 개별소비세, 주세, 부가가치세 및 특례제한법에 따른 세액이 해당 과세기간의 소득금액 계산 시 총수입금액에 반영되지 않는 것을 말합니다. 이는 국세환급가산금, 지방세환급가산금, 이자, 석유판매업자의 세금환급과 같은 경우도 해당됩니다. 이는 세금 환급, 무상으로 받은 자산의 가액 또는 이월된 소득금액을 고려할 때 중요한 사항입니다.
